In [1]:
import sys

!which {sys.executable}

/ut3/jerome/miniconda3/envs/wakai/bin/python


In [2]:
import sys
sys.path.insert(0, '../code')

# TOI catalog

In [3]:
from catalog import get_tois

tois = get_tois(clobber=True)
# tois.columns.tolist()

2025-05-15 13:08:34.159 | INFO     | catalog:get_tois:1393 - Saved: 


In [4]:
tois.tail()

,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Mass (M_Sun) err,Sectors,Date TOI Alerted (UTC),Date TOI Updated (UTC),Date Modified,Comments,ra_deg,dec_deg,GaiaDR3_exofop,GaiaDR3_vizier
6475,259381479,7451.01,NaN,3,4,3,3,1,4,4,...,NaN,"18,58,85",2025-05-01,2025-05-01,2025-05-03 12:20:30,found in faint-star QLP search,43.399500,46.174561,NaN,NaN
6476,327249454,7452.01,NaN,3,4,3,3,1,4,4,...,NaN,"17,18,24,57,58,77,78,84,85",2025-05-01,2025-05-01,2025-05-03 12:20:30,found in faint-star QLP search,3.511750,58.167081,NaN,NaN
6477,347944406,7453.01,NaN,3,4,3,3,1,4,4,...,0.222863,"18,58,85",2025-05-01,2025-05-01,2025-05-03 12:20:30,found in faint-star QLP search,34.003542,57.346072,NaN,NaN
6478,130087198,7454.01,NaN,3,4,3,3,1,4,4,...,NaN,"18,58,85",2025-05-01,2025-05-01,2025-05-03 12:20:30,found in faint-star QLP search,37.777375,40.724964,NaN,NaN
6479,196351790,7455.01,NaN,3,4,3,3,1,4,4,...,NaN,"17,57,58,84,85",2025-05-01,2025-05-01,2025-05-03 12:20:30,found in faint-star QLP search; weak even tran...,14.410875,47.522036,NaN,NaN


In [7]:
tois[tois.Comments.apply(lambda x: 'young' in str(x))][['TOI','Comments']].values

array([[942.01,
        'Probable young K dwarf. Transit is believable enough. Maybe also a signal at 10.15 days with Tc=1447?'],
       [3604.01,
        'V-shaped; found in faint-star QLP search; no stellar radius; possible young or evolved K star'],
       [5743.01,
        'radius possibly underestimated; potentially young host star'],
       [5956.01,
        'V-shaped; large; young host star; ~5 sigma odd-even in SPOC s57'],
       [6258.01,
        'low SNR; asymmetric transit shape; flaring possibly young host'],
       [6678.01,
        'rotational modulation at 7-8 days may indicate young host star']],
      dtype=object)

In [8]:
tois.shape

(6480, 66)

# cross-match TOI with Gaia DR3 using `astroquery.xmatch`

In [9]:
import astropy.units as u
from astroquery.xmatch import XMatch
import pandas as pd

gaia_table = XMatch.query(
    cat1=open('../data/TOIs.csv'),
    cat2="vizier:I/355/gaiadr3",  # The Vizier identifier for Gaia DR3
    max_distance=1 * u.arcsec,    # Set the maximum matching radius
    colRA1="ra_deg",  # Column name for Right Ascension in TIC results
    colDec1="dec_deg" # Column name for Declination in TIC results
)
gaia_df = gaia_table.to_pandas()

# results to 6237 matches
matches = gaia_df.copy()

# Apply magnitude filtering criteria
# For main-sequence stars, the difference between TESS T and Gaia G is around 0–0.5 magnitudes
# Redder stars might have T - G values closer to 1–2 magnitudes.
# For giants or highly reddened stars, the difference could be larger.
if False:
    # results to 5971 matches
    matches = []
    for i, row in gaia_df.iterrows():
        teff=row['Stellar Eff Temp (K)']
        if teff<3500:
            magnitude_threshold = 2
        elif (teff>3500) and (teff<6000):
            magnitude_threshold = 1
        else:
            magnitude_threshold = 1.5
        if abs(row['Gmag'] - row['TESS Mag']) < magnitude_threshold:
            matches.append(row)
    matches = pd.concat(matches, axis=1).T
if False:
    # results to 5700 matches
    magnitude_threshold = 1
    matches = gaia_df[abs(gaia_df['Gmag'] - gaia_df['TESS Mag']) < magnitude_threshold]
if False:
    # results to 4878 matches
    teff_threshold = 1000
    matches = gaia_df[abs(gaia_df['Teff'] - gaia_df['Stellar Eff Temp (K)']) < teff_threshold]
matches.tail()

Could not import regions, which is required for some of the functionalities of this module.


,angDist,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,...,B_AG,E(BP-RP),b_E(BP-RP),B_E(BP-RP),Lib,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,RADEcorJ2000
7740,0.079164,259381479,7451.01,NaN,3,4,3,3,1,4,...,NaN,NaN,NaN,NaN,NaN,43.399531,46.174566,0.595016,0.648654,0.2903
7741,0.006298,327249454,7452.01,NaN,3,4,3,3,1,4,...,0.9097,0.4886,0.4849,0.4927,A,3.511682,58.167092,0.227478,0.218697,-0.4282
7742,0.012678,347944406,7453.01,NaN,3,4,3,3,1,4,...,NaN,NaN,NaN,NaN,NaN,34.003481,57.346115,0.171583,0.213738,0.1890
7743,0.025960,130087198,7454.01,NaN,3,4,3,3,1,4,...,0.1028,0.0495,0.0444,0.0553,MARCS,37.777380,40.724970,0.336897,0.297551,0.0898
7744,0.045442,196351790,7455.01,NaN,3,4,3,3,1,4,...,0.0362,0.0049,0.0015,0.0192,PHOENIX,14.410832,47.522054,0.251246,0.213116,-0.0815


In [10]:
matches.shape

(7745, 204)

In [11]:
matches['GaiaDR3'] = matches['DR3Name'].apply(lambda x: int(x.split()[-1]))
matches.to_csv('../data/TOIs_GaiaDR3_xmatch.csv', index=False)

In [12]:
idx = tois.TOI.isin(matches.TOI.unique())
tois[~idx].shape

(15, 66)

In [13]:
tois[~idx].to_csv('../data/TOIs_GaiaDR3_xmatch_missing.csv', index=False)